In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, pandas as pd
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) "
                     "Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.get("https://www.amazon.com/s?k=car+speakers&rh=p_36%3A4700-7200")

def get_all_links_on_page(driver):
    try:
        elements = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.a-link-normal.s-no-outline"))
        )
        return [el.get_attribute("href") for el in elements if el.get_attribute("href")]
    except:
        return []

def clean_links(links):
    links = list(set([l for l in links if l]))
    links = [l for l in links if "/dp/" in l or "/gp/" in l]
    return links

all_links = []
page = 1

try:
    while True:
        time.sleep(random.uniform(2, 5))

        attempts = 0
        while "Dogs of Amazon" in driver.page_source and attempts < 25:
            print(f"Страница {page}: ошибка Amazon, попытка {attempts+1}")
            driver.refresh()
            time.sleep(random.uniform(3, 6))
            attempts += 1

        if "Dogs of Amazon" in driver.page_source:
            print(f"Страница {page}: пропущена после 25 неудачных попыток")
            break

        page_links = get_all_links_on_page(driver)
        if not page_links:
            print(f"Страница {page}: товаров больше нет, останавливаемся")
            break

        all_links.extend(page_links)
        print(f"Страница {page}: собрано {len(page_links)} ссылок, всего {len(all_links)}")

        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.s-pagination-next")
            if "disabled" in next_button.get_attribute("class"):
                print("Кнопка Next неактивна — достигнут конец списка")
                break
            next_button.click()
            page += 1
        except:
            print("Кнопка Next не найдена — конец списка")
            break
finally:
    driver.quit()

cleaned_links = clean_links(all_links)
df = pd.DataFrame(cleaned_links, columns=["product_link"])
df.to_csv("amazon_car_speakers.csv", index=False, encoding="utf-8")

print(f"\nИтого собрано {len(cleaned_links)} ссылок, сохранено в amazon_car_speakers.csv")
df.head(10)


Страница 1: ошибка Amazon, попытка 1
Страница 1: ошибка Amazon, попытка 2
Страница 1: ошибка Amazon, попытка 3
Страница 1: ошибка Amazon, попытка 4
Страница 1: ошибка Amazon, попытка 5
Страница 1: ошибка Amazon, попытка 6
Страница 1: собрано 20 ссылок, всего 20
Страница 2: собрано 16 ссылок, всего 36
Страница 3: собрано 16 ссылок, всего 52
Страница 4: собрано 16 ссылок, всего 68
Страница 5: собрано 16 ссылок, всего 84
Страница 6: собрано 16 ссылок, всего 100
Страница 7: собрано 16 ссылок, всего 116
Страница 8: собрано 16 ссылок, всего 132
Страница 9: собрано 16 ссылок, всего 148
Страница 10: собрано 16 ссылок, всего 164
Страница 11: собрано 16 ссылок, всего 180
Страница 12: собрано 16 ссылок, всего 196
Страница 13: собрано 16 ссылок, всего 212
Страница 14: собрано 16 ссылок, всего 228
Страница 15: собрано 16 ссылок, всего 244
Страница 16: собрано 16 ссылок, всего 260
Страница 17: собрано 16 ссылок, всего 276
Страница 18: собрано 16 ссылок, всего 292
Страница 19: собрано 16 ссылок, всег

,product_link
0,https://www.amazon.com/BLAUPUNKT-BPS-E683-3-Wa...
1,https://www.amazon.com/GEARit-Gauge-Wire-100ft...
2,https://www.amazon.com/Planet-Audio-AC8D-Voice...
3,https://www.amazon.com/Orion-XTR350HP-Aluminum...
4,https://www.amazon.com/NEW-American-Bass-SQ6-5...
5,https://www.amazon.com/Coupe-Right-Speaker-Cov...
6,https://www.amazon.com/Audiopipe-Woofer-1000W-...
7,https://www.amazon.com/Pieces-Spring-Terminal-...
8,https://www.amazon.com/MB-DS1-254-Discus-Shall...
9,https://www.amazon.com/CERWIN-Vega-Mobile-H740...


In [ ]:
import pandas as pd
import time, random, functools
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import threading
from random import randint

logs = []
user_agent_lock = threading.Lock()

def log_start(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        logs.append({
            "Function": func.__name__,
            "Start": start
        })
        return func(*args, **kwargs)
    return wrapper

def get_new_driver():
    global request_counter
    with user_agent_lock:
        request_counter += 1
        if request_counter % 10 == 0:
            random_version = randint(120, 140)
        else:
            random_version = 120


def get_driver():
    opts = Options()
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--window-size=1920,1080")

    return webdriver.Chrome(options=opts)

@log_start
def parse_product(driver, link):
    driver.get(link)
    time.sleep(random.uniform(2, 4))

    try:
        continue_btn = driver.find_element(By.XPATH, '/html/body/div/div[1]/div[3]/div/div/form/div/div/span/span/button')
        continue_btn.click()
        time.sleep(2)
    except NoSuchElementException:
        pass

    try:
        name = driver.find_element(By.ID, "productTitle").text.strip()
    except:
        name = None

    try:
        type_ = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[5]/div[4]/div[44]/div/table/tbody/tr[1]/td[2]/span').text.strip()
    except:
        type_ = None

    try:
        min_power_raw = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[5]/div[4]/div[44]/div/table/tbody/tr[2]/td[2]/span').text
    except:
        min_power_raw = None

    try:
        max_power_raw = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[5]/div[4]/div[44]/div/table/tbody/tr[3]/td[2]/span').text
    except:
        max_power_raw = None

    try:
        min_frequency_raw = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[5]/div[4]/div[44]/div/table/tbody/tr[4]/td[2]/span').text
    except:
        min_frequency_raw = None

    try:
        max_frequency_raw = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[5]/div[4]/div[44]/div/table/tbody/tr[5]/td[2]/span').text
    except:
        max_frequency_raw = None

    return {
        "Name": name,
        "Type": type_,
        "Brand": min_power_raw,
        "Speaker Maximum": max_power_raw,
        "Connectivity Technology": min_frequency_raw,
        "Audio Output Mode": max_frequency_raw,
        "Mounting Type": link
    }

@log_start
def worker(links, part_id):
    driver = get_driver()
    results = []
    for link in links:
        try:
            results.append(parse_product(driver, link))
        except Exception as e:
            print(f"Ошибка на {link}: {e}")
    driver.quit()

    df = pd.DataFrame(results)
    df = df.where(pd.notnull(df), "None")
    df.to_csv(f"part_{part_id}.csv", index=False, encoding="utf-8-sig")

@log_start
def load_links():
    df_links = pd.read_csv("amazon_car_speakers.csv")
    return df_links["product_link"].tolist()

@log_start
def main():
    all_links = load_links()
    chunks = np.array_split(all_links, 10)

    with ThreadPoolExecutor(max_workers=10) as executor:
        for i, chunk in enumerate(chunks, start=1):
            executor.submit(worker, chunk, i)

    final_df = pd.concat([pd.read_csv(f"part_{i}.csv") for i in range(1, 11)], ignore_index=True)
    final_df.to_csv("final_products.csv", index=False, encoding="utf-8-sig")

    df_logs = pd.DataFrame(logs)
    df_logs.to_csv("log.csv", index=False, encoding="utf-8-sig")

if __name__ == "__main__":
    main()


Нажата кнопка 'Продолжить покупки'
Нажата кнопка 'Продолжить покупки'
Нажата кнопка 'Продолжить покупки'
Нажата кнопка 'Продолжить покупки'
Нажата кнопка 'Продолжить покупки'
Нажата кнопка 'Продолжить покупки'
Нажата кнопка 'Продолжить покупки'
Нажата кнопка 'Продолжить покупки'
